<a href="https://colab.research.google.com/github/VittorioBartolomeoSecondin/DVISProject-CAValli_Team/blob/main/Preprocessing_story3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing libraries and connecting to Google Drive

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import glob
import json
import csv
import requests
import os
import zipfile
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Reading and preprocessing data

In [ ]:
all_files = glob.glob("/content/drive/MyDrive/DV_project/story2/*.xlsx")
all_files.sort()
print(all_files)

['/content/drive/MyDrive/DV_project/story2/early_leavers_by_sex_labour.xlsx', '/content/drive/MyDrive/DV_project/story2/pop_edu_sex_age_absolute.xlsx', '/content/drive/MyDrive/DV_project/story2/pop_edu_sex_age_percentage.xlsx', '/content/drive/MyDrive/DV_project/story2/poverty_exclusion_risk.xlsx', '/content/drive/MyDrive/DV_project/story2/poverty_exclusion_risk_by_age.xlsx', '/content/drive/MyDrive/DV_project/story2/poverty_exclusion_risk_by_education.xlsx']


## Dictionaries: country --> abbreviation (and viceversa)

We create 2 dictionaries linking each country to its short abbreviation (*country_abbreviations*) and viceversa (*country_abbreviations_reversed*).

In [ ]:
country_abbreviations = {
    'European Union - 27 countries (from 2020)': 'EU',
    'Belgium': 'BEL',
    'Bulgaria': 'BGR',
    'Czechia': 'CZE',
    'Denmark': 'DNK',
    'Germany': 'DEU',
    'Estonia': 'EST',
    'Ireland': 'IRL',
    'Greece': 'GRC',
    'Spain': 'ESP',
    'France': 'FRA',
    'Croatia': 'HRV',
    'Italy': 'ITA',
    'Cyprus': 'CYP',
    'Latvia': 'LVA',
    'Lithuania': 'LTU',
    'Luxembourg': 'LUX',
    'Hungary': 'HUN',
    'Malta': 'MLT',
    'Netherlands': 'NLD',
    'Austria': 'AUT',
    'Poland': 'POL',
    'Portugal': 'PRT',
    'Romania': 'ROU',
    'Slovenia': 'SVN',
    'Slovakia': 'SVK',
    'Finland': 'FIN',
    'Sweden': 'SWE',
    'Iceland': 'ISL',
    'Norway': 'NOR',
    'Switzerland': 'CHE',
    'Montenegro': 'MNE',
    'North Macedonia': 'MKD',
    'Serbia': 'SRB',
    'Türkiye': 'TUR',
}

country_abbreviations_reversed = {value: key for key, value in country_abbreviations.items()}

# Download all CSV files

We download all the CSV files we created.

In [ ]:
directory_path = '/content'
file_type = ".csv"
files_to_download = glob.glob(f"{directory_path}/*{file_type}")
zip_filename = "grouped_barcharts.zip"
with zipfile.ZipFile(zip_filename, "w") as zipf:
    for file in files_to_download:
        zipf.write(file, arcname = os.path.basename(file))
files.download(zip_filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

We remove all the CSV files we downloaded.

In [ ]:
for file in files_to_download:
    os.remove(file)